In [ ]:
"""
Created By    : Jared W. Marquis
Creation Date : 01 August 2022
Course        : ATSC 528 - Atmospheric Data Analysis
Assignment    : #02 - Successive Corrections

Purpose:
Script to take sparse upper air observations and analyze them on a
polar stereographic map projection using successive corrections.
[PUT MORE INFORMATION HERE - I.E., WHAT SPECIFIC THING IS BEING DONE]

"""
__author__    = "Jared W. Marquis"
__contact__   = "jared.marquis@und.edu"

In [2]:
### Import Required Modules (shouldn't need to change) ###
import numpy as np                 #numpy for math
import matplotlib.pyplot as plt    #matplotlib for plotting
import cartopy.crs as ccrs         #cartopy for plotting on map
import cartopy.feature as cfeature #cartopy basic shapefiles

In [30]:
### Create function for Cressman Analysis ###
def weights(d,roi):
    if d > roi:
        temp = 0
    else:
        temp = (roi**2 - d**2)/(roi**2 + d**2)
    return temp


In [4]:
### Read in observations ###
aplace = np.loadtxt("RAOBs_201903131200.txt",
                 delimiter=",",usecols=0,dtype='str')
anum = np.genfromtxt("RAOBs_201903131200.txt", delimiter=",", names=("stn","lat", "lon", "hgts", "dir", "spd"))
#display(anum2)
lat = anum['lat']
lon = anum['lon']
height = anum['hgts']

#print(len(anum[:,0]))

In [5]:
### Set up analysis map with a 22x28 rectangular grid of points ###
x0 = 18.9
y0 = -6.3
xgrid = x0+np.arange(22)*1.27
ygrid = y0+np.arange(28)*1.27 #in southwest corner

x,y = np.meshgrid(xgrid, ygrid, indexing='xy')
#print(alist)

In [18]:
### convert obs lat/long to x,y###
'''
Note: to convert between lat/lon, we use the general equation
 y = m*rho*sigma*cos(phi)*sin(lambda0 - lambda), where rho = avg radius of earth (in cm), phi = lat, lambda = lon, lambda0 = 60N (given), m = map conversion factor, and sigma = ((1+sin(phi0))/(1+sin(phi)))
 x = m*rho*sigma*cos(phi)*cos(lambda0 - lambda)
'''
xobs = np.empty(len(lon))
yobs = np.empty(len(lat))
m = 1/15000000
l = -115
phi0 = 60
r = 637100000 #centimeters

for n in np.arange(len(anum)):
    xobs[n] = m*r*((1+np.sin(phi0*np.pi/180))/(1+np.sin(lat[n]*np.pi/180)))*np.cos(lat[n]*np.pi/180)*np.cos((lon[n]*np.pi/180)-(l*np.pi/180))
    yobs[n] = m*r*((1+np.sin(phi0*np.pi/180))/(1+np.sin(lat[n]*np.pi/180)))*np.cos(lat[n]*np.pi/180)*np.sin((lon[n]*np.pi/180)-(l*np.pi/180))


In [19]:
### Perform 500mb geopotential height analyses using a Cressman weighting Function###
#Use radii of influence 4, 2.5, 1.5 *dmin

### Finding dmin ###
'''
radius = (xobs**2 + yobs**2)**0.5
dmin = sum(radius)/len(radius)
'''
dmin = 0
#radius = np.zeros((len(xobs),len(yobs)))
tdmin = np.empty((len(xobs),len(yobs)))
for i in range(len(xobs)):
    relx = xobs[i] - xobs
    rely = yobs[i] - yobs
    radius = (relx**2 + rely**2)**0.5
    radius = np.delete(radius,i)
    dmin += np.amin(radius)
    
dmin = dmin / len(xobs)

print(dmin)


2.5548700820071426


In [15]:
print(y[0,0],y[1,0],y[0,1])
print(x[0,0],x[1,0],x[0,1])

-6.3 -5.029999999999999 -6.3
18.9 18.9 20.169999999999998


In [32]:
tempx=np.empty(len(xobs))
tempy=np.empty(len(yobs))
radius=np.empty(shape=(len(x[0]),len(x),len(xobs)))

for i in range(len(x[0])):
    for j in range(len(x)):
        for k in range(len(xobs)):
            tempx[k] = x[j,i] - xobs[k]
            tempy[k] = y[j,i] - yobs[k]
            #determine number of points in ROI
            radius[i,j,k] = (tempx[k]**2 + tempy[k]**2)**0.5

w15 = np.zeros(shape=(len(x),len(x[0]),len(xobs)))
w25 = np.zeros(shape=(len(x),len(x[0]),len(xobs)))
w40 = np.zeros(shape=(len(x),len(x[0]),len(xobs)))

for i in range(len(x)):
    for j in range(len(x[0])):
        for k in range(len(xobs)):
            w15[i,j,k] = weights(radius[j,i,k],1.5*dmin)
            w25[i,j,k] = weights(radius[j,i,k],2.5*dmin)
            w40[i,j,k] = weights(radius[j,i,k],4*dmin)  

print(w15)

[[[0.         0.         0.         ... 0.30820212 0.         0.88228993]
  [0.         0.         0.         ... 0.32287342 0.         0.85816285]
  [0.         0.         0.         ... 0.16648812 0.         0.5269842 ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.         0.         0.         ... 0.         0.         0.52969493]
  [0.         0.         0.         ... 0.         0.         0.51372197]
  [0.         0.         0.         ... 0.         0.         0.28643347]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.         0.         0.         ... 0.         0.         0.12866317]
  [0.         0.      

In [33]:
print(w40)

[[[0.         0.         0.03193635 ... 0.86156523 0.         0.98256519]
  [0.         0.         0.12445385 ... 0.8657058  0.         0.97875964]
  [0.         0.         0.2120729  ... 0.81738328 0.         0.91651364]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.         0.         0.11847882 ... 0.73130256 0.         0.91711325]
  [0.         0.         0.22798819 ... 0.73488337 0.         0.91355459]
  [0.         0.         0.3332403  ... 0.69302395 0.         0.85528291]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.         0.         0.19827272 ... 0.57862947 0.         0.80413644]
  [0.         0.      

In [ ]:
### First analysis, no successive corrections ###
'''
fa(ri) = fb(ri) + {SUM(w(ob-an)*[fo(ob loc) - fb(ob loc)]}/[SUM(w(ob-an))]
analysis at analysis point = background at analysis point + {sum of weights*(ob-an)*[obs at obs location - background at obs location]}/
                                                                                    {SUM[weights(ob-an)]}
'''



In [ ]:
### Next Analyses ###

In [ ]:
### Calculate Analysis Differences


In [ ]:
### Plot 500mb analyses over a map ###
#use old code...



In [ ]:
### Plot Analysis Differences ###


In [ ]:
### Store the analyses in text files ###


In [ ]:
### Store the difference fields in text files ###


In [ ]:
### Store RMS values in text file ###


In [ ]:
### In a separte text file (or below), answer the following questions ###
'''
1 - Describe the general features that you see in your contoured analyses.
    

2 - Describe the differences that you see in your contoured analyses.  
    Does one analysis seem to be smoother than the other?  If so, what would cause this?
    

3 - What happens as you increase the number of successive correction passes?  Is this 
    desirable?  Why or why not?
    

'''